In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
!mkdir Train
!mkdir Test

!mkdir Train/Fake
!mkdir Train/Real

!mkdir Test/Fake
!mkdir Test/Real

In [3]:
# Repositioning the train data so that keras can read it.

PATH = "/kaggle/input/signature-verification-dataset/sign_data/train/"

for i in os.listdir(PATH):
    
    contol = i.split("_")
    
    try:
    
        if contol[1]=="forg":
            
            os.system("cp -r {} Train/Fake".format(PATH+i))
            
    except:
        
        os.system("cp -r {} Train/Real".format(PATH+i))
        

In [4]:
# Rearrange the test data in such a way that keras can read it.

PATH = "/kaggle/input/signature-verification-dataset/sign_data/test/"

for i in os.listdir(PATH):
    
    contol = i.split("_")
    
    try:
    
        if contol[1]=="forg":
            
            os.system("cp -r {} Test/Fake".format(PATH+i))
            
    except:
        
        os.system("cp -r {} Test/Real".format(PATH+i))

In [5]:
# locations

train_dir = os.path.join("/kaggle/working/Train") 

test_dir = os.path.join("/kaggle/working/Test") 

In [6]:
train_datagen = ImageDataGenerator(

      # rescaling pixels between 0,1
      rescale=1./255,

      # Angle of random rotation of images in degrees (0-180)
      rotation_range=40,

      # horizontal and vertical scrolling ratios of images
      width_shift_range=0.2,

      # horizontal and vertical scrolling ratios of images
      height_shift_range=0.2,

      # sprain operation
      shear_range=0.2,

      # zoom operation
      zoom_range=0.2,

      # rotate image vertically
      horizontal_flip=True,

      # excess after processing
      # determines how the image points are filled
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(

        # target directory
        train_dir,

        # all images will be resized as (150x150)
        target_size=(200, 200),

        # batch or stack size
        batch_size=64,

        # binary tags required
        # because we are using binary_crossentropy
        class_mode='binary')

In [8]:
test_generator = test_datagen.flow_from_directory(

        test_dir,

        target_size=(200, 200),

        batch_size=64,

        class_mode='binary')

In [9]:
plt.imshow(train_generator[0][0][5])
print("Label : ",train_generator[0][1][5])

In [10]:
plt.imshow(train_generator[0][0][60])
print("Label : ",train_generator[0][1][60])

In [11]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    
    self.cnn1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(3,200,200))
    self.cnn2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
    self.cnn3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')
    
    self.flatten = tf.keras.layers.Flatten()
    
    self.dense1 = tf.keras.layers.Dense(512, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    
    x = self.cnn1(inputs)
    x = self.cnn2(x)
    x = self.cnn3(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

model = MyModel()

In [12]:
input_shape = (None, 200, 200, 3)
model.build(input_shape)
model.summary()

In [13]:
model.compile(
    # loss function
    loss="binary_crossentropy",
    
    # Optimization:
    # Considering the loss created by the data, which is the input of our network
    # self-update mechanism
    optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),

    # metrics to follow during training and testing. 
    metrics=["acc"])

In [ ]:
# We will get the acc, loss, val_acc, val_loss values from the variable named history.
history = model.fit_generator(

    # training data
    train_generator,

    # the number of samples it will run through until the loop finishes (stack to get)
    steps_per_epoch=train_generator.samples//train_generator.batch_size,

    # number of cycles
    epochs= 40,

    verbose=2)


In [ ]:
# Training accuracy score
acc = history.history["acc"]

# training loss score
loss = history.history["loss"]

# We will plot graphs according to the number of epochs.
epochs = range(1, len(acc) + 1)

# We had training data drawn for itself..
plt.plot(epochs, acc, "bo", label="accuracy")

# the title of graph
plt.title("accuracy")

plt.legend()

plt.figure()

# we had training data drawn for itself.
plt.plot(epochs, loss, "bo", label="loss")


# the title of our graph
plt.title("loss")

plt.legend()

# display on screen
plt.show()

In [ ]:
# Loss and verification with test data
model.evaluate(test_generator)

In [ ]:
plt.imshow(test_generator[0][0][5])
print("Label : ",test_generator[0][1][5])

test_input = test_generator[0][0][5]
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)

pred = model.predict(test_input)

if pred>=0.5:
    pred = 1
    
else:
    
    pred = 0
    
print("Predict : ",float(pred))

In [ ]:
plt.imshow(test_generator[0][0][30])
print("Label : ",test_generator[0][1][30])

test_input = test_generator[0][0][30]
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)

pred = model.predict(test_input)

if pred>=0.5:
    pred = 1
    
else:
    
    pred = 0
    
print("Predict : ",float(pred))